In [93]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import math
import time
import os

In [95]:
city_url = 'https://geographic.org/streetview/usa/ca/orange/index.html'
response = requests.get(city_url)
soup = BeautifulSoup(response.text, 'html.parser')
cities = soup.body.ul.find_all('li')
clean_cities = []
for city in cities:
    clean_cities.append(f'{city.text}, CA')
print(clean_cities)

In [240]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = f'https://www.redfin.com/city/9361/CA/Irvine/filter/property-type=house,status=active'
browser.visit(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\adams\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [105]:
state = 'CA'
for city in clean_cities:
    browser.fill('searchInputBox',f'{city}\n')
    time.sleep(4) #Delay is necessary but time can be adjusted
    try:
        browser.find_by_id('download-and-save').click()
    except:
        try:
            browser.find_by_xpath('/html/body/div[4]/div/div[2]/div/div/div/div[2]/div/div/div[2]/div[1]/div/div[2]').click()
            browser.find_by_id('download-and-save').click()
        except:
            continue
    browser.fill('searchInputBox','')
    browser.find_by_xpath('/html/body/div[1]/div[3]/div/div/header[2]/div[1]/div[2]/div/form/div[1]/div/div/div/div/input').click()



In [106]:
!mv ~/Downloads/redfin* ~/gitTemp/FinalProject/Data/Dirty

In [127]:
path_to_data = './Data'
file_list = os.listdir(path_to_data)
for file in file_list:
    temp_csv = pd.read_csv(f'{path_to_data}/{file}')
    temp_df = pd.DataFrame(temp_csv)
    try:
        redfin_df = redfin_df.append(temp_df)
    except:
        redfin_df = temp_df
redfin_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [141]:
#Only take houses currently listed
redfin_df = redfin_df[redfin_df['SALE TYPE'] == 'MLS Listing']

Single Family Residential    3868
Name: PROPERTY TYPE, dtype: int64

In [142]:
del redfin_df['SOLD DATE']
del redfin_df['SALE TYPE']
del redfin_df['NEXT OPEN HOUSE END TIME']
del redfin_df['NEXT OPEN HOUSE START TIME']
del redfin_df['SOURCE']
del redfin_df['FAVORITE']
del redfin_df['INTERESTED']
del redfin_df['STATUS']
del redfin_df['HOA/MONTH']
del redfin_df['PROPERTY TYPE']
#Location is unnecessary as street, city, state, and zipcode are all included in other columns 
del redfin_df['LOCATION']

In [154]:
#Remove redundant listings
redfin_df = redfin_df.drop_duplicates()
redfin_df.reset_index(inplace=True)
del redfin_df['index']

In [276]:
listing_url = redfin_df['URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)']
house_url = []
school_name = []
school_rating = []
school_type = []
school_grades = []
average_rating = []

In [283]:
#They don't like webscraping so I have to manually resume scraping when it hits a recaptcha
begin = 637

for i in range(begin, len(listing_url)):
    rating = 0
    browser.visit(listing_url[i])
    soup = BeautifulSoup(browser.html, 'html.parser')
    try:
        school_info = soup.find('div', class_='schools-content').table.find_all('tr')
        for school in school_info:
            house_url.append(listing_url[i])
            try:
                school_name.append(school.find('div', class_='school-name').text)
            except:
                school_name.append('issue')
            try:
                school_rating.append(float(school.find("span", class_="rating-num").text))
            except:
                school_rating.append('issue')
            try:
                rating += float(school.find("span", class_="rating-num").text)
            except:
                rating = 'issue'
            try:
                school_serves = school.find('div', class_='sub-info').text.split(' • ')
                try:
                    school_type.append(school_serves[0])
                except:
                    school_type.append('issue')
                try:
                    school_grades.append(school_serves[1])
                except:
                    school_grades.append('issue')
            except:
                school_type.append('issue')
                school_grades.append('issue')
        try:
            average_rating.append(round(rating / len(school_info), 3))
        except:
            average_rating.append('issue')
    except:
        try:
            ghost_town = soup.find('div', class_='GhostTown').text
            if ghost_town == "We couldn't find any schools that service this home or are within 5 miles of this home.":
                average_rating.append('No Schools')
        except:
            break
    print(i)

637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886


In [287]:
redfin_df['Average School Rating'] = average_rating

<ipython-input-287-a85e63c3b8ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redfin_df['Average School Rating'] = average_rating


In [300]:
clean_redfin = redfin_df[redfin_df['Average School Rating'] != 'issue']
clean_redfin.reset_index(inplace = True)
del clean_redfin['index']
clean_redfin.to_csv('./Data/Clean_Real_Estate.csv')

In [296]:
school_df = pd.DataFrame({
    'Listing URL': house_url,
    'School Name': school_name,
    'School Rating': school_rating,
    'School Type': school_type,
    'School Grades': school_grades
})
school_df.to_csv('./Data/School_Data.csv')

In [295]:
#Probably need to drop the 605 columns with School Rating issues but left for now
school_df[school_df['School Rating'] != 'issue']

Listing URL  \
0     http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...   
1     http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...   
2     http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...   
3     http://www.redfin.com/CA/Laguna-Beach/11-Sande...   
4     http://www.redfin.com/CA/Laguna-Beach/11-Sande...   
...                                                 ...   
5606  http://www.redfin.com/CA/Yorba-Linda/4115-Live...   
5607  http://www.redfin.com/CA/Yorba-Linda/4115-Live...   
5608  http://www.redfin.com/CA/Yorba-Linda/20360-Flo...   
5609  http://www.redfin.com/CA/Yorba-Linda/20360-Flo...   
5610  http://www.redfin.com/CA/Yorba-Linda/20360-Flo...   

                    School Name School Rating School Type School Grades  
0     Don Juan Avila Elementary             8      Public        K to 5  
1         Don Juan Avila Middle             8      Public        6 to 8  
2             Aliso Niguel High             9      Public       9 to 12  
3           El Morro Elementary             9      Public        K to 5  
4               Thurston Middle             9      Public        6 to 8  
...                         ...           ...         ...           ...  
5606      Bernardo Yorba Middle             8      Public        7 to 8  
5607           Yorba Linda High            10      Public       9 to 12  
5608  Mabel M. Paine Elementary             8      Public        K to 5  
5609         Yorba Linda Middle             9      Public        6 to 8  
5610           Yorba Linda High            10      Public       9 to 12  

[5006 rows x 5 columns]

In [301]:
clean_redfin

ADDRESS         CITY STATE OR PROVINCE  \
0             11 Hazelbranch  Aliso Viejo                CA   
1           11 Sanderling Ln  Aliso Viejo                CA   
2                  5 Skygate  Aliso Viejo                CA   
3             15 Starling Ln  Aliso Viejo                CA   
4          78 Summerland Cir  Aliso Viejo                CA   
...                      ...          ...               ...   
1346        5191 Wendover Rd  Yorba Linda                CA   
1347  5461 Mountain View Ave  Yorba Linda                CA   
1348       18950 Suncrest Dr  Yorba Linda                CA   
1349        4115 Live Oak Ln  Yorba Linda                CA   
1350       20360 Florence Rd  Yorba Linda                CA   

      ZIP OR POSTAL CODE    PRICE  BEDS  BATHS  SQUARE FEET  LOT SIZE  \
0                  92656   975000     3   2.50       1470.0    4950.0   
1                  92656   850000     3   2.50       1393.0    3000.0   
2                  92656  1350000     4   3.00       2918.0    5040.0   
3                  92656   995000     3   2.50       1605.0    3480.0   
4                  92656  1549000     4   4.50       3258.0    5546.0   
...                  ...      ...   ...    ...          ...       ...   
1346               92886  1974000     7   5.25       3463.0   18270.0   
1347               92886  1087000     4   3.00       1803.0    9600.0   
1348               92886  3250000     5   5.50       5364.0   23681.0   
1349               92886  4799999     6   7.50       9302.0   43560.0   
1350               92886  2996666     5   5.50       5349.0   19668.0   

      YEAR BUILT  DAYS ON MARKET  $/SQUARE FEET  \
0         1993.0             3.0          663.0   
1         1989.0             3.0          610.0   
2         1989.0             5.0          463.0   
3         1989.0             6.0          620.0   
4         2011.0            10.0          475.0   
...          ...             ...            ...   
1346      1965.0            87.0          570.0   
1347      1961.0            99.0          603.0   
1348      2002.0           104.0          606.0   
1349      1991.0           125.0          516.0   
1350      2013.0           138.0          560.0   

     URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)  \
0     http://www.redfin.com/CA/Aliso-Viejo/11-Hazelb...                                           
1     http://www.redfin.com/CA/Laguna-Beach/11-Sande...                                           
2     http://www.redfin.com/CA/Aliso-Viejo/5-Skygate...                                           
3     http://www.redfin.com/CA/Aliso-Viejo/15-Starli...                                           
4     http://www.redfin.com/CA/Aliso-Viejo/78-Summer...                                           
...                                                 ...                                           
1346  http://www.redfin.com/CA/Yorba-Linda/5191-Wend...                                           
1347  http://www.redfin.com/CA/Yorba-Linda/5461-Moun...                                           
1348  http://www.redfin.com/CA/Yorba-Linda/18950-Sun...                                           
1349  http://www.redfin.com/CA/Yorba-Linda/4115-Live...                                           
1350  http://www.redfin.com/CA/Yorba-Linda/20360-Flo...                                           

            MLS#   LATITUDE   LONGITUDE Average School Rating  
0     OC21181915  33.592613 -117.738368                 8.333  
1     LG21181169  33.592240 -117.749226                     9  
2     OC21160272  33.594179 -117.740063                 8.333  
3     LG21176453  33.596696 -117.744324                     9  
4     CV21176676  33.595652 -117.723661                 7.667  
...          ...        ...         ...                   ...  
1346  PW21107820  33.885782 -117.797048                 8.333  
1347  OC21104129  33.881948 -117.804192                 8.667  
1348  OC2